< [The PCRaster Dynamic Modelling Framework](PCRasterDynamicModellingFramework.ipynb) | [Contents](Contents.ipynb) | [Building the dynamic model](STREAM_Dynamic.ipynb) >

# 3. Preparation de la section `initiale` du modèle
Un modèle dynamique avec le cadre Python du PCRaster a une section "initiale" et une "dynamique". Comme indiqué précédemment, la section initiale contient du code qui ne doit être exécuté qu'une seule fois pour initialiser le modèle. Il ne doit pas être itéré dans le temps.
## 3.1 Lire des cartes statiques
Nous devons initialiser le modèle STREAM en lisant les cartes statiques suivantes dans la section « initial » :
`` `Python
landuse.map # carte d'occupation des sols
sol.map # carte des sols
metstat.map # carte avec les stations météorologiques
dem.map # Modèle numérique d'élévation
mask.map # cloner la carte
`` `

Ajoutez les lignes à la section "initiale" du script pour lire les cartes statiques et exécutez le script pour vérifier s'il n'y a pas d'erreurs. Notez que nous utilisons `landuse.map` et `sol.map` uniquement dans la section `initial` pour créer des cartes avec des tableaux de correspondance. Nous n'utilisons pas non plus `dem.map` dans la section` dynamic`. Toutes les autres cartes sont utilisées à la fois dans les sections `initial`et`dynamic`. Réfléchissez donc à la façon de créer des variables locales et globales.

In [ ]:
from pcraster import *
from pcraster.framework import *

class RunoffModel(DynamicModel):
    def __init__(self, cloneMap):
        DynamicModel.__init__(self)
        setclone(cloneMap)
    
    def initial(self):
        # Ajoutez ici les cartes statiques dont nous avons besoin pour lire à partir du disque
        landuse = self.readmap("./Data/landuse")
        soil = self.readmap("./Data/soil")
        
        
        # Ajoutez ici les tableaux de correspondance
        self.InterceptionThreshold = lookupscalar("./Data/d.tbl",landuse)
        self.report(self.InterceptionThreshold,"d")
    
    def dynamic(self):
        Precipitation = self.readmap("./Data/pr") 
        
myModel = RunoffModel("./Data/mask.map")
dynModelFw = DynamicFramework(myModel, lastTimeStep=10, firstTimestep=1)
dynModelFw.run()

## 3.2 Définir les paramètres (constantes)
Nous devons également définir les paramètres du modèle (constantes) dans la section initiale. Ces constantes seront utilisées dans la section « dynamique », nous devons donc les rendre globales. Ils sont ajoutés ci-dessous avec dans les commentaires leur signification. Plus tard dans la section "dynamique", il deviendra plus clair comment ces constantes seront utilisées.

In [1]:
from pcraster import *
from pcraster.framework import *

class RunoffModel(DynamicModel):
    def __init__(self, cloneMap):
        DynamicModel.__init__(self)
        setclone(cloneMap)
    
    def initial(self):
        # Ajoutez ici les cartes statiques dont nous avons besoin pour lire à partir du disque
        landuse = self.readmap("./Data/landuse")
        soil = self.readmap("./Data/soil")
        self.metstat = self.readmap("./Data/metstat")
        DEM = self.readmap("./Data/dem")
        self.mask = self.readmap("./Data/mask")
        
        # Ajoutez ici les constantes en tant que variables globales
        self.Ku = scalar(1.5)                  # Constante de récession de l'écoulement de la zone non saturée à la zone saturée
        self.surface = scalar(0.01)            # surface d'une cellule de grille (km2)
        self.ConvConst = scalar(0.00001157407) # De mm/10 days à m3/s: *1/1000 * 1/10 *
                                               # 1/24 * 1/3600 * 100^2
        self.rtq = scalar(1.2)                 # constante de Débit rapide de récession
        self.rts = scalar(5.3)                 # constante de Débit lent de récession
        self.Su = scalar(50.0)                 # zone de stockage non saturée
        self.Ssmax = scalar(0.0)               # paramètre contrôlant le débit des eaux souterraines vers la rivière
        self.Ss = scalar(50.0)                 # zone de stockage saturée
        
        
        # Ajoutez ici les tableaux de correspondance
        self.InterceptionThreshold = lookupscalar("./Data/d.tbl",landuse)
        self.report(self.InterceptionThreshold,"d")
    
    def dynamic(self):
        Precipitation = self.readmap("./Data/pr") 
        
myModel = RunoffModel("./Data/mask.map")
dynModelFw = DynamicFramework(myModel, lastTimeStep=10, firstTimestep=1)
dynModelFw.run()

ModuleNotFoundError: No module named 'pcraster'

## 3.3 Créer des entrées à partir de tableaux de correspondance
Maintenant, nous ajoutons les autres paramètres pour lesquels nous utilisons des tableaux de correspondance

Pour la carte des sols, il s'agit de :
* Stockage maximum de la zone non saturée 'SuMax' (mm)
* Coefficient de débit rapide (-)

Nous utilisons des valeurs hypothétiques et des classes de sols.

Pour la carte d'occupation des sols, il s'agit de :
* Le coefficient de séparation (-)
* Élévation capillaire maximale (mm)

## 3.4 Créer la carte de direction des écoulements
Nous créons également la carte de direction d´écoulement dans la section `initial`. Ajoutez le code à la ligne 46 et exécutez le script.

In [ ]:
from pcraster import *
from pcraster.framework import *

class RunoffModel(DynamicModel):
    def __init__(self, cloneMap):
        DynamicModel.__init__(self)
        setclone(cloneMap)
    
    def initial(self):
        # Ajoutez ici les cartes statiques dont nous avons besoin pour lire à partir du disque
        landuse = self.readmap("./Data/landuse")
        soil = self.readmap("./Data/soil")
        self.metstat = self.readmap("./Data/metstat")
        DEM = self.readmap("./Data/dem")
        self.mask = self.readmap("./Data/mask")
        
        # Ajoutez ici les constantes en tant que variables globales
        self.Ku = scalar(1.5)                  # Constante de récession de l'écoulement de la zone non saturée à la zone saturée
        self.surface = scalar(0.01)            # surface d'une cellule de grille (km2)
        self.ConvConst = scalar(0.00001157407) # De mm/10 days à m3/s: *1/1000 * 1/10 *
                                               # 1/24 * 1/3600 * 100^2
        self.rtq = scalar(1.2)                 # constante de Débit rapide de récession
        self.rts = scalar(5.3)                 # constante de Débit lent de récession
        self.Su = scalar(50.0)                 # zone de stockage non saturée
        self.Ssmax = scalar(0.0)               # paramètre contrôlant le débit des eaux souterraines vers la rivière
        self.Ss = scalar(50.0)                 # zone de stockage saturée
        
        
        # Ajoutez ici les tableaux de correspondance
        self.InterceptionThreshold = lookupscalar("./Data/d.tbl",landuse)   # seuil d´interception (mm)
        self.report(self.InterceptionThreshold,"./Data/d")
        
        self.SuMax = lookupscalar("./Data/smax.tbl",soil)                   # stockage maximal zone non saturée (mm)
        self.report(self.SuMax,"./Data/SuMax")
        
        self.SeparationCoefficient = lookupscalar("./Data/cr.tbl",landuse)  # coefficient de séparation (-)
        self.report(self.SeparationCoefficient,"./Data/cr")
        
        self.QuickFlowCoefficient = lookupscalar("./Data/qc.tbl",soil)      # Coefficient d'écoulement rapide (-)
        self.report(self.QuickFlowCoefficient,"./Data/Qc")
            
        self.MaxCapRise = lookupscalar("./Data/cp.tbl",landuse)             # potentiel de remontée capillaire (mm)
        self.report(self.MaxCapRise,"./Data/cmax")
        
        # Ajoutez ici le calcul de la carte de direction d´écoulement
   
    
    def dynamic(self):
        Precipitation = self.readmap("./Data/pr") 
        
myModel = RunoffModel("./Data/mask.map")
dynModelFw = DynamicFramework(myModel, lastTimeStep=10, firstTimestep=1)
dynModelFw.run()

Maintenant, cela prend plus de temps pour exécuter le modèle. Mais ce qui est bien, c'est qu'il n'a besoin de générer la carte de direction d´écoulement qu'une seule fois. Dans le futur, vous pourriez même vouloir lire la carte de direction d´écoulement à partir du disque au lieu de la calculer à chaque fois que vous exécutez le modèle.

La carte de direction d´écoulement doit être ajustée au masque et rester hydrologiquement correcte. Nous utilisons l'opération [lddmask operation](https://pcraster.geo.uu.nl/pcraster/4.3.0/documentation/pcraster_manual/sphinx/op_lddmask.html).

Le code a été ajouté ci-dessous et le résultat est enregistré sur le disque.

In [ ]:
from pcraster import *
from pcraster.framework import *

class RunoffModel(DynamicModel):
    def __init__(self, cloneMap):
        DynamicModel.__init__(self)
        setclone(cloneMap)
    
    def initial(self):
         # Ajoutez ici les cartes statiques dont nous avons besoin pour lire à partir du disque
        landuse = self.readmap("./Data/landuse")
        soil = self.readmap("./Data/soil")
        self.metstat = self.readmap("./Data/metstat")
        DEM = self.readmap("./Data/dem")
        self.mask = self.readmap("./Data/mask")
        
        
       # Ajoutez ici les constantes en tant que variables globales
        self.Ku = scalar(1.5)                  # Constante de récession de l'écoulement de la zone non saturée à la zone saturée
        self.surface = scalar(0.01)            # surface d'une cellule de grille (km2)
        self.ConvConst = scalar(0.00001157407) # De mm/10 days à m3/s: *1/1000 * 1/10 *
                                               # 1/24 * 1/3600 * 100^2
        self.rtq = scalar(1.2)                 # constante de Débit rapide de récession
        self.rts = scalar(5.3)                 # constante de Débit lent de récession
        self.Su = scalar(50.0)                 # zone de stockage non saturée
        self.Ssmax = scalar(0.0)               # paramètre contrôlant le débit des eaux souterraines vers la rivière
        self.Ss = scalar(50.0)                 # zone de stockage saturée
        
        
        # Ajoutez ici les tableaux de correspondance
        self.InterceptionThreshold = lookupscalar("./Data/d.tbl",landuse)   # seuil d´interception (mm)
        self.report(self.InterceptionThreshold,"./Data/d")
        
        self.SuMax = lookupscalar("./Data/smax.tbl",soil)                   # stockage maximal zone non saturée (mm)
        self.report(self.SuMax,"./Data/SuMax")
        
        self.SeparationCoefficient = lookupscalar("./Data/cr.tbl",landuse)  # coefficient de séparation (-)
        self.report(self.SeparationCoefficient,"./Data/cr")
        
        self.QuickFlowCoefficient = lookupscalar("./Data/qc.tbl",soil)      # Coefficient d'écoulement rapide (-)
        self.report(self.QuickFlowCoefficient,"./Data/Qc")
            
        self.MaxCapRise = lookupscalar("./Data/cp.tbl",landuse)             # potentiel de remontée capillaire (mm)
        self.report(self.MaxCapRise,"./Data/cmax")
        
        # Ajoutez ici le calcul de la carte de direction d´écoulement
        self.flowdirection = lddcreate(DEM,1e31,1e31,1e31,1e31)
        self.flowdirection = lddmask(self.flowdirection,self.mask)
        self.report(self.flowdirection,"./Data/ldd")
 
    
    def dynamic(self):
        Precipitation = self.readmap("./Data/pr") 
        
myModel = RunoffModel("./Data/mask.map")
dynModelFw = DynamicFramework(myModel, lastTimeStep=10, firstTimestep=1)
dynModelFw.run()

Nous pouvons maintenant procéder à la construction du modèle dynamique dans la section suivante.

< [The PCRaster Dynamic Modelling Framework](PCRasterDynamicModellingFramework.ipynb) | [Contents](Contents.ipynb) | [Building the dynamic model](STREAM_Dynamic.ipynb) >